<a href="https://colab.research.google.com/github/gremlin97/ToySatCLIP/blob/main/ToySatCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm
!pip install transformers
!pip install torchgeo
!pip install geoclip
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 4.8 MB/s eta 0:00:00


In [2]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from google.colab import drive
from PIL import Image
import pandas as pd
import numpy as np
import torchgeo.models
from torchgeo.models import ResNet18_Weights
import rasterio
import torchvision.transforms as transforms
import itertools
import shutil

import torch
from torch import nn
import torch.nn.functional as F
import timm
from geoclip import LocationEncoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<ipython-input-2-511e4b496f97>:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
!mkdir -p data && cd data && mkdir s2
%cd /content/data/s2

/content/data/s2


In [4]:
!wget https://satclip.z13.web.core.windows.net/satclip/index.csv
df = pd.read_csv('/content/data/s2/index.csv')

--2024-04-15 00:09:05--  https://satclip.z13.web.core.windows.net/satclip/index.csv
Resolving satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)... 52.239.221.231
Connecting to satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)|52.239.221.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5318595 (5.1M) [text/csv]
Saving to: ‘index.csv’

index.csv           100%[===================>]   5.07M  1.36MB/s    in 3.7s    

2024-04-15 00:09:10 (1.36 MB/s) - ‘index.csv’ saved [5318595/5318595]



In [5]:
# If you want to sample the datapoints
num_samples = 1000
random_indices = np.random.choice(len(df), num_samples, replace=False)
sampled_df = df.iloc[random_indices]
sampled_df.to_csv('sampled_output.csv', index=False)

In [6]:
# You can download the whole/partial dataset (stop the cell)
!mkdir images
%cd images
!wget https://satclip.z13.web.core.windows.net/satclip/satclip.tar

/content/data/s2/images
--2024-04-15 00:09:10--  https://satclip.z13.web.core.windows.net/satclip/satclip.tar
Resolving satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)... 52.239.221.231
Connecting to satclip.z13.web.core.windows.net (satclip.z13.web.core.windows.net)|52.239.221.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83874291712 (78G) [application/x-tar]
Saving to: ‘satclip.tar’

satclip.tar          10%[=>                  ]   8.12G  13.2MB/s    eta 92m 57s^C


In [7]:
!tar -xf satclip.tar

tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now


In [8]:
df = df.set_index('fn')

In [9]:
def list_files(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.append(filepath)
    return files

directory_path = '/content/data/s2/images/'
files_in_directory = list_files(directory_path)

train_dict = {}
train_dict['lat'] = []
train_dict['lon'] = []
train_dict['image'] = []
for x in files_in_directory:
  if x.endswith('.tif'):
    x = (x.split('/')[5])
    train_dict['lat'].append(df.loc[x, 'lat'])
    train_dict['lon'].append(df.loc[x, 'lon'])
    train_dict['image'].append(x)

In [10]:
df_train = pd.DataFrame(train_dict)

In [11]:
print(df_train)

             lat         lon            image
0     -71.357831   49.808834  patch_14199.tif
1      25.477850   17.048403    patch_139.tif
2     -10.189076  -69.210487  patch_17597.tif
3      28.414298    7.531442   patch_1162.tif
4      82.714755  -20.838712  patch_16486.tif
...          ...         ...              ...
10387   5.344652   16.893933  patch_16346.tif
10388   9.353308   19.585405   patch_1576.tif
10389  54.654688   78.475036  patch_17694.tif
10390  35.293712   39.293925  patch_17900.tif
10391 -75.553206  111.237536  patch_13194.tif

[10392 rows x 3 columns]


In [12]:
class Params:
    debug = False
    batch_size = 32*5
    num_workers = 18
    head_lr = 1e-3
    image_encoder_lr = 1e-5
    location_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_path = '/content/data/s2/images'
    model_name = 'resnet50'
    image_embedding = 512
    location_embedding = 512
    pretrained = True
    trainable = True
    temperature = 1.0
    size = 224
    num_projection_layers = 1
    projection_dim = 1024
    dropout = 0.1

In [13]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, images, lat, lon, transforms):
        self.images = images
        self.lat = list(lat)
        self.lon = list(lon)
        self.transforms = transforms

    def __getitem__(self, idx):
        it = {}

        with rasterio.open(f"{Params.image_path}/{self.images[idx]}") as f:
          image = f.read().astype(np.float32)
          ext = np.expand_dims(image[0], axis=0)
          image = np.vstack((image, ext))

        image = torch.from_numpy(image)
        image = self.transforms(image)

        it['image'] = image
        it['lat'] = self.lat[idx]
        it['lon'] = self.lon[idx]
        return it

    def __len__(self):
        return len(self.lat)

def tf(mode="train"):
    if mode == "train":
        return transforms.Compose([
            transforms.Resize((Params.size, Params.size)),
            transforms.Normalize(mean=[0.5] * 13, std=[0.5] * 13)
        ])
    else:
        return transforms.Compose([
            transforms.Resize((Params.size, Params.size)),
            transforms.Normalize(mean=[0.5] * 13, std=[0.5] * 13)
        ])

In [15]:
class Resnet(nn.Module):
    def __init__(
        self, trainable=Params.trainable
    ):
        super().__init__()
        weights = ResNet18_Weights.SENTINEL2_ALL_MOCO
        in_chans = weights.meta["in_chans"]
        self.visual = timm.create_model("resnet18", in_chans=in_chans, num_classes=0)
        self.visual.load_state_dict(weights.get_state_dict(progress=True), strict=False)
        self.visual.requires_grad_(False)
        self.visual.fc.requires_grad_(True)
        # for p in self.visual.parameters():
        #   p.requires_grad = trainable

    def forward(self, x):
        return self.visual(x)

In [16]:
class LE(nn.Module):

  def __init__(self):
    super().__init__()
    self.encoder = LocationEncoder()

    for p in self.encoder.parameters():
      p.requires_grad = Params.trainable

  def forward(self, a):
    c = a[0][0]
    d = a[0][1]
    merged_list = [[lat.item(), long.item()] for lat, long in zip(c, d)]
    merged_list = torch.tensor(merged_list)
    merged_list = merged_list.to(device)
    embed = self.encoder(merged_list)
    return embed

In [17]:
class Project(nn.Module):
    def __init__(self, input_dim, output_dim=Params.projection_dim, dropout_prob=Params.dropout):
        super().__init__()
        self.projection_layer = nn.Linear(input_dim, output_dim)
        self.fc_layer = nn.Linear(output_dim, output_dim)
        self.layer_norm = nn.LayerNorm(output_dim)
        self.gelu_activation = nn.GELU()
        self.dropout_layer = nn.Dropout(dropout_prob)

    def forward(self, x):
        projected = self.projection_layer(x)
        x = self.gelu_activation(projected)
        x = self.fc_layer(x)
        x = self.dropout_layer(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [18]:
class SatCLIP(nn.Module):
    def __init__(self, temperature=Params.temperature, ie=Params.image_embedding, le=Params.location_embedding,):
        super().__init__()
        self.image_encoder = Resnet()
        self.location_encoder = LE()
        self.image_projection = Project(input_dim=ie)
        self.loc_projection = Project(input_dim=le)
        self.temperature = temperature

    def forward(self, batch):
        image_features = self.image_encoder(batch["image"])
        loc = [[batch['lat'], batch['lon']]]
        loc_features = self.location_encoder(loc)
        image_embeddings = self.image_projection(image_features)
        loc_embeddings = self.loc_projection(loc_features)

        logits = (loc_embeddings @ image_embeddings.T) / self.temperature
        image_similarity_matrix = image_embeddings @ image_embeddings.T
        loc_similarity_matrix = loc_embeddings @ loc_embeddings.T
        targets = F.softmax((image_similarity_matrix + loc_similarity_matrix) / 2 * self.temperature, dim=-1)
        loc_loss = calculate_cross_entropy(logits, targets)
        images_loss = calculate_cross_entropy(logits.T, targets.T)
        loss = (images_loss + loc_loss) / 2.0
        return loss.mean()

def calculate_cross_entropy(predictions, targets):
    log_softmax_func = nn.LogSoftmax(dim=-1)
    individual_losses = (-targets * log_softmax_func(predictions)).sum(1)
    return individual_losses

In [19]:
def add_id_column(df):
    df['id'] = range(1, len(df) + 1)

def split_train_validation(df):
    max_custom_id = df["id"].max() + 1
    all_custom_ids = np.arange(0, max_custom_id)
    validation_custom_ids = np.random.choice(
        all_custom_ids, size=int(0.1 * len(all_custom_ids)), replace=False
    )
    training_custom_ids = [id_ for id_ in all_custom_ids if id_ not in validation_custom_ids]
    train_data = df[df["id"].isin(training_custom_ids)].reset_index(drop=True)
    valid_data = df[df["id"].isin(validation_custom_ids)].reset_index(drop=True)
    return train_data, valid_data

def create_data_loader(df, mode):
    transforms = tf(mode=mode)
    dataset = Dataset(df["image"].values, df["lat"].values, df['lon'].values, transforms=transforms)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=Params.batch_size)
    return data_loader

add_id_column(df_train)

In [20]:
def train_model_one_epoch(model, data_loader, optimizer, lr_scheduler=None, scheduler_step="batch"):
    train_loss_meter = AvgMeter()
    progress_bar = tqdm(data_loader, total=len(data_loader))
    for batch in progress_bar:
        batch = {key: value.to(device) for key, value in batch.items()}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if scheduler_step == "batch" and lr_scheduler is not None:
            lr_scheduler.step()

        batch_size = batch["image"].size(0)
        train_loss_meter.update(loss.item(), batch_size)

        progress_bar.set_postfix(train_loss=train_loss_meter.avg, lr=get_lr(optimizer))
    return train_loss_meter


def validate_model_one_epoch(model, data_loader):
    val_loss_meter = AvgMeter()
    progress_bar = tqdm(data_loader, total=len(data_loader))
    for batch in progress_bar:
        batch = {key: value.to(device) for key, value in batch.items()}
        loss = model(batch)
        print('Validation Loss:', loss)
        batch_size = batch["image"].size(0)
        val_loss_meter.update(loss.item(), batch_size)
        progress_bar.set_postfix(valid_loss=val_loss_meter.avg)
    return val_loss_meter

In [21]:
def main():
    global train_df
    global valid_df
    train_df, valid_df = load_data(df_train)
    train_loader = create_data_loader(train_df, mode="train")
    valid_loader = create_data_loader(valid_df, mode="valid")

    model = initialize_model()
    optimizer = setup_optimizer(model)
    lr_scheduler = setup_lr_scheduler(optimizer)
    best_loss = float('inf')

    for epoch in range(Params.epochs):
        print('Training')
        print(f"Epoch: {epoch + 1}")
        train_loss = train(model, train_loader, optimizer, lr_scheduler)
        valid_loss = evaluate(model, valid_loader)

        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            save_model(model, "satclip.pt")
            print("Best Model is saved!")

        lr_scheduler.step(valid_loss.avg)

def load_data(df_train):
    train_df, valid_df = split_train_validation(df_train)
    return train_df, valid_df

def initialize_model():
    model = SatCLIP().to(Params.device)
    return model

def setup_optimizer(model):
    image_encoder_params = {"params": model.image_encoder.parameters(), "lr": Params.image_encoder_lr}
    location_encoder_params = {"params": model.location_encoder.parameters(), "lr": Params.location_encoder_lr}
    projection_params = {
        "params": itertools.chain(model.image_projection.parameters(), model.loc_projection.parameters()),
        "lr": Params.head_lr,
        "weight_decay": Params.weight_decay
    }
    params = [image_encoder_params, location_encoder_params, projection_params]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    return optimizer

def setup_lr_scheduler(optimizer):
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=Params.patience, factor=Params.factor
    )
    return lr_scheduler

def train(model, train_loader, optimizer, lr_scheduler):
    model.train()
    step = "epoch"
    train_loss = train_model_one_epoch(model, train_loader, optimizer, lr_scheduler, step)
    return train_loss

def evaluate(model, data_loader):
    model.eval()
    with torch.no_grad():
        valid_loss = validate_model_one_epoch(model, data_loader)
    return valid_loss

def save_model(model, filename):
    torch.save(model.state_dict(), filename)

In [25]:
main()

Training
Epoch: 1


  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Validation Loss: tensor(6.7682, device='cuda:0')
Validation Loss: tensor(6.5403, device='cuda:0')
Validation Loss: tensor(6.4647, device='cuda:0')
Validation Loss: tensor(6.5110, device='cuda:0')
Validation Loss: tensor(6.5306, device='cuda:0')
Validation Loss: tensor(6.5088, device='cuda:0')
Validation Loss: tensor(6.5220, device='cuda:0')
Best Model is saved!
Training
Epoch: 2


  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

Validation Loss: tensor(4.1388, device='cuda:0')
Validation Loss: tensor(4.1191, device='cuda:0')
Validation Loss: tensor(4.1999, device='cuda:0')
Validation Loss: tensor(4.1717, device='cuda:0')
Validation Loss: tensor(4.1177, device='cuda:0')
Validation Loss: tensor(4.2375, device='cuda:0')
Validation Loss: tensor(3.4598, device='cuda:0')
Best Model is saved!
Training
Epoch: 5


  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Validation Loss: tensor(4.0136, device='cuda:0')
Validation Loss: tensor(3.9844, device='cuda:0')
Validation Loss: tensor(4.0429, device='cuda:0')
Validation Loss: tensor(4.0379, device='cuda:0')
Validation Loss: tensor(3.9732, device='cuda:0')
Validation Loss: tensor(4.1057, device='cuda:0')
Validation Loss: tensor(3.3484, device='cuda:0')
Best Model is saved!


In [24]:
# # if you find that an image does not exist run this command for the particular image
# index_to_delete = df_train[df_train['image'] == 'patch_1935.tif'].index[0]
# df_train = df_train.drop(index_to_delete)

In [26]:
model = SatCLIP()
checkpoint = torch.load("/content/data/s2/images/satclip.pt")
model.load_state_dict(checkpoint)
model.eval()

SatCLIP(
  (image_encoder): Resnet(
    (visual): ResNet(
      (conv1): Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act1): ReLU(inplace=True)
          (aa): Identity()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
        )
        (1): BasicBlock(
          (conv1): 

In [27]:
valid_loader = create_data_loader(valid_df, mode="valid")
model_path = '/content/data/s2/images/satclip.pt'
model = SatCLIP().to(Params.device)
model.load_state_dict(torch.load(model_path, map_location=Params.device))
model.eval()

valid_image_embeddings = []
valid_loc_embeddings = []
with torch.no_grad():
    for batch in tqdm(valid_loader):
        image_features = model.image_encoder(batch["image"].to(Params.device))
        image_embeddings = model.image_projection(image_features)
        valid_image_embeddings.append(image_embeddings)
        loc = [[batch['lat'], batch['lon']]]
        loc_features = model.location_encoder(loc)
        loc_embeddings = model.loc_projection(loc_features)
        valid_loc_embeddings.append(loc_embeddings)

  0%|          | 0/7 [00:00<?, ?it/s]

In [28]:
valid_image_embeddings

[tensor([[ 0.3973, -1.1235, -0.0221,  ...,  0.1329,  0.0400,  0.6138],
         [ 0.5207, -0.9490,  0.1140,  ...,  0.1673, -0.0530,  0.5422],
         [ 0.3978, -1.2261,  0.2946,  ...,  0.4206, -0.0898,  0.5466],
         ...,
         [ 0.4101, -1.1111,  0.0045,  ...,  0.5407, -0.0513,  0.5567],
         [ 0.4672, -1.1110,  0.0280,  ...,  0.4362, -0.0883,  0.5420],
         [ 0.4400, -1.0216,  0.2055,  ...,  0.3387, -0.0607,  0.4461]],
        device='cuda:0'),
 tensor([[ 0.3329, -1.0030, -0.0392,  ...,  0.4086, -0.0267,  0.5564],
         [ 0.3673, -1.1963,  0.0653,  ...,  0.2826, -0.1009,  0.5415],
         [ 0.1926, -1.1812, -0.0543,  ...,  0.3953,  0.1787,  0.5757],
         ...,
         [ 0.3035, -1.0709, -0.0146,  ...,  0.2901,  0.0374,  0.5962],
         [ 0.3942, -1.1881, -0.0770,  ...,  0.3093, -0.0176,  0.5844],
         [ 0.3860, -1.1341, -0.0944,  ...,  0.4433, -0.0583,  0.6006]],
        device='cuda:0'),
 tensor([[ 0.1788, -1.0307, -0.0484,  ...,  0.1909,  0.0205,  0.55

In [29]:
valid_loc_embeddings

[tensor([[-0.0263,  0.0035,  0.4482,  ..., -0.2081, -0.0345,  0.0952],
         [ 0.1429, -0.0565,  0.6073,  ..., -0.0132, -0.2038,  0.0645],
         [ 0.0739, -0.1100,  0.7892,  ..., -0.0565, -0.2256,  0.0020],
         ...,
         [ 0.1494, -0.0193,  0.5601,  ...,  0.0544, -0.2065,  0.0037],
         [ 0.0721, -0.0889,  0.6698,  ...,  0.1031, -0.2376,  0.0809],
         [ 0.1443, -0.0187,  0.7492,  ..., -0.1018, -0.2448,  0.0065]],
        device='cuda:0'),
 tensor([[ 0.0087,  0.0582,  0.3768,  ..., -0.0254, -0.1653,  0.1754],
         [ 0.0029, -0.1767,  0.3358,  ..., -0.0103, -0.1385,  0.1544],
         [ 0.0335, -0.1065,  0.3050,  ..., -0.0980, -0.1906,  0.1453],
         ...,
         [-0.1766, -0.1788,  0.3165,  ..., -0.2583, -0.1039,  0.1678],
         [-0.0630, -0.1949,  0.2645,  ..., -0.1645, -0.1234,  0.2389],
         [-0.0643, -0.1395,  0.2019,  ...,  0.0086, -0.0788,  0.2123]],
        device='cuda:0'),
 tensor([[-0.1149, -0.2199,  0.4251,  ..., -0.1955, -0.0885,  0.15

In [30]:
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
shutil.copy('/content/data/s2/images/satclip.pt', '/content/drive/MyDrive/')

'/content/drive/MyDrive/satclip.pt'

In [33]:
# import os
# file_path = path
# file_size_bytes = os.path.getsize(file_path)
# file_size_mb = file_size_bytes / (1024 * 1024)
# print(f"File size: {file_size_mb:.2f} MB")